In [1]:
import sys, os, csv
sys.path.append('..')
from datetime import datetime
import numpy as np
import pandas as pd
import h5py

from tools.common import monitor_memory
import nasdaqdatalink as ndl


In [2]:
#del sys.modules['tools.instruments']
#del sys.modules['tools.visualisation']
from tools.instruments import instruments
from tools.visualisation import view
from tools.constants import SRF_CONTRACTS_DB_PATH

In [4]:
quotes=ndl.get('SRF/CME_ADU2008')

In [6]:
quotes['2008-05-15':]

,Open,High,Low,Settle,Volume,Prev. Day Open Interest
Date,,,,,,
2008-05-15,0.9173,0.9268,0.9146,0.9261,69.0,1112.0
2008-05-16,0.9270,0.9406,0.9255,0.9387,1224.0,1732.0
2008-05-19,0.9401,0.9416,0.9362,0.9388,325.0,3159.0
2008-05-20,0.9375,0.9465,0.9371,0.9434,66.0,3179.0
2008-05-21,0.9437,0.9501,0.9408,0.9473,242.0,3191.0
...,...,...,...,...,...,...
2008-09-09,0.8138,0.8172,0.8000,0.8055,1315.0,72478.0
2008-09-10,0.7982,0.8128,0.7975,0.8027,2946.0,67749.0
2008-09-11,0.7966,0.8071,0.7896,0.7998,1299.0,57788.0


In [3]:
monitor_memory(dir())

[('np', 157), ('pd', 65), ('ndl', 29), ('instruments', 6), ('SRF_CONTRACTS_DB_PATH', 5), ('monitor_memory', 3), ('view', 3)]


In [8]:
file = h5py.File(SRF_CONTRACTS_DB_PATH, 'a')

In [4]:
# 저장할 DB파일 생성 및 그룹생성
file = h5py.File(SRF_CONTRACTS_DB_PATH, 'w')
for instrument in instruments.values():
    if instrument.contracts:
        file.create_group(instrument.symbol)

In [21]:
# 다운 가능한 상품에 대해서 nasdaq-data-link 에서 SRF 데이터 다운
dtypes= np.dtype([
        ('date', 'i'),
        ('open', 'f'),
        ('high','f'),
        ('low','f'),
        ('close','f'),
        ('volume', 'i'),
        ('open_interest','i')
    ])

i=0
for instrument in instruments.values():
    if not instrument.contracts:
        continue
        
    if len(file[instrument.symbol]):
        print(f"SKIP DB: {instrument.name}")
        continue
    
    for contract in instrument.contracts:
        print(f"{i}: {contract[6]} 다운로드 중...")
        symbol = instrument.symbol
        code = contract[2]
        data = ndl.get(f'SRF/{code}')
        data.reset_index(inplace=True)
        data['Date'] = data['Date'].values.astype('M8[D]').astype('i')
        
        #미결제 약정에 들어있는 nan value를 0으로 변경
        if data.iloc[:,6].isna().sum():
            counter = data.iloc[:,6].isna().sum()
            print(f"미결제 약정 Nan Value 발생: {contract[6]} ({counter}개)\n")
            data.iloc[:,6] = data.iloc[:,6].fillna(0)
        
        #거래량에 들어있는 nan value를 0으로 변경
        if data.iloc[:,5].isna().sum():
            counter = data.iloc[:,5].isna().sum()
            print(f"거래량 Nan Value 발생: {contract[5]}({counter}개)")
            data.iloc[:,5] = data.iloc[:,5].fillna(0)
        
        data = np.array(list(map(tuple, data.values)), dtypes)
        file[symbol].create_dataset(code, data=np.array(data, dtypes))
        i+= 1 #카운터
    
print("완료")

SKIP DB: CME Australian Dollar AUD
SKIP DB: CBOT Soybean Oil
SKIP DB: CME British Pound GBP
SKIP DB: CBOT Corn
SKIP DB: ICE Cocoa
SKIP DB: CME Canadian Dollar CAD
SKIP DB: NYMEX WTI Crude Oil
SKIP DB: ICE Cotton
SKIP DB: ICE US Dollar Index
SKIP DB: CME Euro FX
SKIP DB: CME Eurodollar
SKIP DB: CME S&P 500 Index E-Mini
SKIP DB: CME 30 Day Federal Funds Futures
SKIP DB: CBOT 5-year US Treasury Note
SKIP DB: ICE Gasoil
SKIP DB: NYMEX Gold
SKIP DB: COMEX Copper
SKIP DB: NYMEX Heating Oil
SKIP DB: CME Japanese Yen JPY
SKIP DB: ICE Coffee C
SKIP DB: CME Kansas City Wheat
SKIP DB: CME Live Cattle
0: CME Lean Hogs, February 1970 (LNG1970) 다운로드 중...
1: CME Lean Hogs, April 1970 (LNJ1970) 다운로드 중...
2: CME Lean Hogs, June 1970 (LNM1970) 다운로드 중...
3: CME Lean Hogs, July 1970 (LNN1970) 다운로드 중...
4: CME Lean Hogs, August 1970 (LNQ1970) 다운로드 중...
5: CME Lean Hogs, October 1970 (LNV1970) 다운로드 중...
6: CME Lean Hogs, December 1970 (LNZ1970) 다운로드 중...
7: CME Lean Hogs, February 1971 (LNG1971) 다운로드 중...
8

149: CME Lean Hogs, June 1991 (LNM1991) 다운로드 중...
150: CME Lean Hogs, July 1991 (LNN1991) 다운로드 중...
151: CME Lean Hogs, August 1991 (LNQ1991) 다운로드 중...
152: CME Lean Hogs, October 1991 (LNV1991) 다운로드 중...
153: CME Lean Hogs, December 1991 (LNZ1991) 다운로드 중...
154: CME Lean Hogs, February 1992 (LNG1992) 다운로드 중...
155: CME Lean Hogs, April 1992 (LNJ1992) 다운로드 중...
156: CME Lean Hogs, June 1992 (LNM1992) 다운로드 중...
157: CME Lean Hogs, July 1992 (LNN1992) 다운로드 중...
158: CME Lean Hogs, August 1992 (LNQ1992) 다운로드 중...
159: CME Lean Hogs, October 1992 (LNV1992) 다운로드 중...
160: CME Lean Hogs, December 1992 (LNZ1992) 다운로드 중...
161: CME Lean Hogs, February 1993 (LNG1993) 다운로드 중...
162: CME Lean Hogs, April 1993 (LNJ1993) 다운로드 중...
163: CME Lean Hogs, June 1993 (LNM1993) 다운로드 중...
164: CME Lean Hogs, July 1993 (LNN1993) 다운로드 중...
165: CME Lean Hogs, August 1993 (LNQ1993) 다운로드 중...
166: CME Lean Hogs, October 1993 (LNV1993) 다운로드 중...
167: CME Lean Hogs, December 1993 (LNZ1993) 다운로드 중...
168: CME Lean

307: CME Lean Hogs, December 2013 (LNZ2013) 다운로드 중...
308: CME Lean Hogs, February 2014 (LNG2014) 다운로드 중...
309: CME Lean Hogs, April 2014 (LNJ2014) 다운로드 중...
310: CME Lean Hogs, June 2014 (LNM2014) 다운로드 중...
311: CME Lean Hogs, July 2014 (LNN2014) 다운로드 중...
312: CME Lean Hogs, August 2014 (LNQ2014) 다운로드 중...
313: CME Lean Hogs, October 2014 (LNV2014) 다운로드 중...
314: CME Lean Hogs, December 2014 (LNZ2014) 다운로드 중...
315: CME Lean Hogs, February 2015 (LNG2015) 다운로드 중...
316: CME Lean Hogs, April 2015 (LNJ2015) 다운로드 중...
317: CME Lean Hogs, May 2015 (LNK2015) 다운로드 중...
318: CME Lean Hogs, June 2015 (LNM2015) 다운로드 중...
319: CME Lean Hogs, July 2015 (LNN2015) 다운로드 중...
320: CME Lean Hogs, August 2015 (LNQ2015) 다운로드 중...
321: CME Lean Hogs, October 2015 (LNV2015) 다운로드 중...
322: CME Lean Hogs, December 2015 (LNZ2015) 다운로드 중...
323: CME Lean Hogs, February 2016 (LNG2016) 다운로드 중...
324: CME Lean Hogs, April 2016 (LNJ2016) 다운로드 중...
325: CME Lean Hogs, May 2016 (LNK2016) 다운로드 중...
326: CME Lean H

450: CME S&P 400 Midcap Index, June 2008 (MDM2008) 다운로드 중...
451: CME S&P 400 Midcap Index, September 2008 (MDU2008) 다운로드 중...
452: CME S&P 400 Midcap Index, December 2008 (MDZ2008) 다운로드 중...
453: CME S&P 400 Midcap Index, March 2009 (MDH2009) 다운로드 중...
454: CME S&P 400 Midcap Index, June 2009 (MDM2009) 다운로드 중...
455: CME S&P 400 Midcap Index, September 2009 (MDU2009) 다운로드 중...
456: CME S&P 400 Midcap Index, December 2009 (MDZ2009) 다운로드 중...
457: CME S&P 400 Midcap Index, March 2010 (MDH2010) 다운로드 중...
458: CME S&P 400 Midcap Index, June 2010 (MDM2010) 다운로드 중...
459: CME S&P 400 Midcap Index, September 2010 (MDU2010) 다운로드 중...
460: CME S&P 400 Midcap Index, December 2010 (MDZ2010) 다운로드 중...
461: CME S&P 400 Midcap Index, March 2011 (MDH2011) 다운로드 중...
462: CME S&P 400 Midcap Index, June 2011 (MDM2011) 다운로드 중...
463: CME S&P 400 Midcap Index, September 2011 (MDU2011) 다운로드 중...
464: CME S&P 400 Midcap Index, December 2011 (MDZ2011) 다운로드 중...
465: CME S&P 400 Midcap Index, March 2012 (MDH

587: Mexican Peso Futures, December 2021 (MPZ2021) 다운로드 중...
588: Mexican Peso Futures, January 2022 (MPF2022) 다운로드 중...
589: Mexican Peso Futures, February 2022 (MPG2022) 다운로드 중...
590: Mexican Peso Futures, March 2022 (MPH2022) 다운로드 중...
591: Mexican Peso Futures, April 2022 (MPJ2022) 다운로드 중...
592: Mexican Peso Futures, May 2022 (MPK2022) 다운로드 중...
593: Mexican Peso Futures, June 2022 (MPM2022) 다운로드 중...
594: Mexican Peso Futures, July 2022 (MPN2022) 다운로드 중...
595: Mexican Peso Futures, August 2022 (MPQ2022) 다운로드 중...
596: Mexican Peso Futures, September 2022 (MPU2022) 다운로드 중...
597: Mexican Peso Futures, October 2022 (MPV2022) 다운로드 중...
598: Mexican Peso Futures, November 2022 (MPX2022) 다운로드 중...
599: Mexican Peso Futures, December 2022 (MPZ2022) 다운로드 중...
600: Mexican Peso Futures, January 2023 (MPF2023) 다운로드 중...
601: Mexican Peso Futures, February 2023 (MPG2023) 다운로드 중...
602: Mexican Peso Futures, March 2023 (MPH2023) 다운로드 중...
603: Mexican Peso Futures, April 2023 (MPJ2023) 다운

...

5018: CBOT 10-year US Treasury Note, December 1993 (TYZ1993) 다운로드 중...
5019: CBOT 10-year US Treasury Note, March 1994 (TYH1994) 다운로드 중...
5020: CBOT 10-year US Treasury Note, June 1994 (TYM1994) 다운로드 중...
5021: CBOT 10-year US Treasury Note, September 1994 (TYU1994) 다운로드 중...
5022: CBOT 10-year US Treasury Note, December 1994 (TYZ1994) 다운로드 중...
5023: CBOT 10-year US Treasury Note, March 1995 (TYH1995) 다운로드 중...
5024: CBOT 10-year US Treasury Note, June 1995 (TYM1995) 다운로드 중...
5025: CBOT 10-year US Treasury Note, September 1995 (TYU1995) 다운로드 중...
5026: CBOT 10-year US Treasury Note, December 1995 (TYZ1995) 다운로드 중...
5027: CBOT 10-year US Treasury Note, March 1996 (TYH1996) 다운로드 중...
5028: CBOT 10-year US Treasury Note, June 1996 (TYM1996) 다운로드 중...
5029: CBOT 10-year US Treasury Note, September 1996 (TYU1996) 다운로드 중...
5030: CBOT 10-year US Treasury Note, December 1996 (TYZ1996) 다운로드 중...
5031: CBOT 10-year US Treasury Note, March 1997 (TYH1997) 다운로드 중...
5032: CBOT 10-year US Treas

5138: CBOT 30-year US Treasury Bond, September 1978 (USU1978) 다운로드 중...
5139: CBOT 30-year US Treasury Bond, December 1978 (USZ1978) 다운로드 중...
5140: CBOT 30-year US Treasury Bond, March 1979 (USH1979) 다운로드 중...
5141: CBOT 30-year US Treasury Bond, June 1979 (USM1979) 다운로드 중...
5142: CBOT 30-year US Treasury Bond, September 1979 (USU1979) 다운로드 중...
5143: CBOT 30-year US Treasury Bond, December 1979 (USZ1979) 다운로드 중...
5144: CBOT 30-year US Treasury Bond, March 1980 (USH1980) 다운로드 중...
5145: CBOT 30-year US Treasury Bond, June 1980 (USM1980) 다운로드 중...
5146: CBOT 30-year US Treasury Bond, September 1980 (USU1980) 다운로드 중...
5147: CBOT 30-year US Treasury Bond, December 1980 (USZ1980) 다운로드 중...
5148: CBOT 30-year US Treasury Bond, March 1981 (USH1981) 다운로드 중...
5149: CBOT 30-year US Treasury Bond, June 1981 (USM1981) 다운로드 중...
5150: CBOT 30-year US Treasury Bond, September 1981 (USU1981) 다운로드 중...
5151: CBOT 30-year US Treasury Bond, December 1981 (USZ1981) 다운로드 중...
5152: CBOT 30-year US T

5256: CBOT 30-year US Treasury Bond, March 2008 (USH2008) 다운로드 중...
5257: CBOT 30-year US Treasury Bond, June 2008 (USM2008) 다운로드 중...
5258: CBOT 30-year US Treasury Bond, September 2008 (USU2008) 다운로드 중...
5259: CBOT 30-year US Treasury Bond, December 2008 (USZ2008) 다운로드 중...
5260: CBOT 30-year US Treasury Bond, March 2009 (USH2009) 다운로드 중...
5261: CBOT 30-year US Treasury Bond, June 2009 (USM2009) 다운로드 중...
5262: CBOT 30-year US Treasury Bond, September 2009 (USU2009) 다운로드 중...
5263: CBOT 30-year US Treasury Bond, December 2009 (USZ2009) 다운로드 중...
5264: CBOT 30-year US Treasury Bond, March 2010 (USH2010) 다운로드 중...
5265: CBOT 30-year US Treasury Bond, June 2010 (USM2010) 다운로드 중...
5266: CBOT 30-year US Treasury Bond, September 2010 (USU2010) 다운로드 중...
5267: CBOT 30-year US Treasury Bond, December 2010 (USZ2010) 다운로드 중...
5268: CBOT 30-year US Treasury Bond, March 2011 (USH2011) 다운로드 중...
5269: CBOT 30-year US Treasury Bond, June 2011 (USM2011) 다운로드 중...
5270: CBOT 30-year US Treasury 

5400: CBOT Wheat, September 1976 (WU1976) 다운로드 중...
5401: CBOT Wheat, December 1976 (WZ1976) 다운로드 중...
5402: CBOT Wheat, March 1977 (WH1977) 다운로드 중...
5403: CBOT Wheat, May 1977 (WK1977) 다운로드 중...
5404: CBOT Wheat, July 1977 (WN1977) 다운로드 중...
5405: CBOT Wheat, September 1977 (WU1977) 다운로드 중...
5406: CBOT Wheat, December 1977 (WZ1977) 다운로드 중...
5407: CBOT Wheat, March 1978 (WH1978) 다운로드 중...
5408: CBOT Wheat, May 1978 (WK1978) 다운로드 중...
5409: CBOT Wheat, July 1978 (WN1978) 다운로드 중...
5410: CBOT Wheat, September 1978 (WU1978) 다운로드 중...
5411: CBOT Wheat, December 1978 (WZ1978) 다운로드 중...
5412: CBOT Wheat, March 1979 (WH1979) 다운로드 중...
5413: CBOT Wheat, May 1979 (WK1979) 다운로드 중...
5414: CBOT Wheat, July 1979 (WN1979) 다운로드 중...
5415: CBOT Wheat, September 1979 (WU1979) 다운로드 중...
5416: CBOT Wheat, December 1979 (WZ1979) 다운로드 중...
5417: CBOT Wheat, March 1980 (WH1980) 다운로드 중...
5418: CBOT Wheat, May 1980 (WK1980) 다운로드 중...
5419: CBOT Wheat, July 1980 (WN1980) 다운로드 중...
5420: CBOT Wheat, Septem

5568: CBOT Wheat, May 2010 (WK2010) 다운로드 중...
5569: CBOT Wheat, July 2010 (WN2010) 다운로드 중...
5570: CBOT Wheat, September 2010 (WU2010) 다운로드 중...
5571: CBOT Wheat, December 2010 (WZ2010) 다운로드 중...
5572: CBOT Wheat, March 2011 (WH2011) 다운로드 중...
5573: CBOT Wheat, May 2011 (WK2011) 다운로드 중...
5574: CBOT Wheat, July 2011 (WN2011) 다운로드 중...
5575: CBOT Wheat, September 2011 (WU2011) 다운로드 중...
5576: CBOT Wheat, December 2011 (WZ2011) 다운로드 중...
5577: CBOT Wheat, March 2012 (WH2012) 다운로드 중...
5578: CBOT Wheat, May 2012 (WK2012) 다운로드 중...
5579: CBOT Wheat, July 2012 (WN2012) 다운로드 중...
5580: CBOT Wheat, September 2012 (WU2012) 다운로드 중...
5581: CBOT Wheat, December 2012 (WZ2012) 다운로드 중...
5582: CBOT Wheat, March 2013 (WH2013) 다운로드 중...
5583: CBOT Wheat, May 2013 (WK2013) 다운로드 중...
5584: CBOT Wheat, July 2013 (WN2013) 다운로드 중...
5585: CBOT Wheat, September 2013 (WU2013) 다운로드 중...
5586: CBOT Wheat, December 2013 (WZ2013) 다운로드 중...
5587: CBOT Wheat, March 2014 (WH2014) 다운로드 중...
5588: CBOT Wheat, May 20

In [7]:
file.close()

### 데이터 확인

In [28]:
from tools.visualisation import view

In [27]:
instruments['W'].contracts

[('W',
  'CME_W',
  'CME_WZ1959',
  datetime.datetime(1959, 7, 1, 0, 0),
  datetime.datetime(1959, 12, 21, 0, 0),
  '2015-04-27 16:17',
  'CBOT Wheat, December 1959 (WZ1959)'),
 ('W',
  'CME_W',
  'CME_WH1960',
  datetime.datetime(1959, 7, 1, 0, 0),
  datetime.datetime(1960, 3, 22, 0, 0),
  '2015-04-27 16:17',
  'CBOT Wheat, March 1960 (WH1960)'),
 ('W',
  'CME_W',
  'CME_WK1960',
  datetime.datetime(1959, 7, 1, 0, 0),
  datetime.datetime(1960, 5, 19, 0, 0),
  '2015-04-27 16:17',
  'CBOT Wheat, May 1960 (WK1960)'),
 ('W',
  'CME_W',
  'CME_WN1960',
  datetime.datetime(1959, 7, 1, 0, 0),
  datetime.datetime(1960, 7, 20, 0, 0),
  '2015-04-27 16:17',
  'CBOT Wheat, July 1960 (WN1960)'),
 ('W',
  'CME_W',
  'CME_WU1960',
  datetime.datetime(1959, 7, 1, 0, 0),
  datetime.datetime(1960, 9, 21, 0, 0),
  '2015-04-27 16:17',
  'CBOT Wheat, September 1960 (WU1960)'),
 ('W',
  'CME_W',
  'CME_WZ1960',
  datetime.datetime(1960, 1, 4, 0, 0),
  datetime.datetime(1960, 12, 20, 0, 0),
  '2015-04-27 16

In [42]:
df = pd.DataFrame(file['W']['CME_WZ1973'][:])

In [43]:
df

,date,open,high,low,close,volume,open_interest
0,1097,239.25,240.00,233.75,234.75,320,275
1,1098,234.25,239.75,234.25,239.25,310,375
2,1099,240.00,241.50,236.25,236.50,410,705
3,1100,237.25,239.50,234.50,236.50,70,740
4,1103,235.75,237.25,235.50,236.50,185,665
...,...,...,...,...,...,...,...
238,1441,572.00,578.00,547.00,550.50,3095,7250
239,1442,550.00,560.00,530.50,544.00,3565,5950
240,1443,547.00,547.00,527.00,528.50,1955,4805
241,1446,535.00,548.50,524.00,548.50,1835,3755
